# Implementing ResNets

In the final exercise, we will implementing a ResNet model! As we did in the previous exercise, we will also load our model with pre-trained weights!

We will be making two models: **ResNet18** and **ResNet50**. The architectural design can be found below

<img src='imgs/resnet.jpg'>

Remember, what makes ResNets so special is the skip connection design in each layer block.

<img src='imgs/skip_connection.png'>

I'll provide most of the necessary code, and you can fill out the rest. First, lets define two functions that will return 3x3 convolution and 1x1 convolutions

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torchvision.models.utils import load_state_dict_from_url 

In [ ]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In the `BasicBlock` class, we will define the layer block used in ResNet 34 or lower. Remember, ResNet34 has the same amount of block repetitions as ResNet50, except that each block in ResNet50 has one extra convolution layer. This layer block will be called `Bottleneck`.

>**Exercise**: Fill out the forward pass of `BasicBlock`. Although it is not clear in the diagram above, make sure to apply a Batch Norm layer after every convolution (before RELU).

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    # Fill out the forward pass!
    def forward(self, x):
        

        return out

>**Exercise**: Fill out the forward pass of `Bottleneck`. Although it is not clear in the diagram above, make sure to apply a Batch Norm layer after every convolution (before RELU).

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4  # Multiplier for dimension of last conv layer in each block

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    # fill out this part!
    def forward(self, x):
        

        return out

Now, we will collect the building blocks to create the full ResNet model

>**Exercise**: Fill out the forward method of the ResNet model.

In [ ]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        replace_stride_with_dilation = [False, False, False]
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def forward(self, x):
        
        return x

Now that we've made our model, we can load some pre-trained weights

In [ ]:
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth'
}

In [ ]:
def make_resnet18_50(arch='resnet18', pretrained=True, progress=True, **kwargs):
    if arch=='resnet18':
        layers = [2, 2, 2, 2]
        block = BasicBlock
    else:
        assert(arch=='resnet50')
        layers = [3, 4, 6, 3]
        block = Bottleneck
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
        model.load_state_dict(state_dict)
    return model

In [ ]:
model = make_resnet18_50('resnet18')
model

In [ ]:
model = make_resnet18_50('resnet50')
model

Now, we will run some tests to make sure that your model's output is the same as the official Pytorch model's output!.

In [ ]:
model = make_resnet18_50('resnet18')
torch_model = models.resnet18(pretrained=True)

rand_input = torch.randn(1, 3, 224, 224)
my_model_output = model(rand_input)
torch_model_output = torch_model(rand_input)
print(my_model_output.equal(torch_model_output))

In [ ]:
model = make_resnet18_50('resnet50')
torch_model = models.resnet50(pretrained=True)

rand_input = torch.randn(1, 3, 224, 224)
my_model_output = model(rand_input)
torch_model_output = torch_model(rand_input)
print(my_model_output.equal(torch_model_output))